In [1]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random
import io

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant, TruncatedNormal
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D, Flatten
import tensorflow.keras.backend as K

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report
from keras_preprocessing.text import tokenizer_from_json

import nltk
from nltk.tokenize import word_tokenize

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.regularizers import l1_l2


In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
embeddings = ["w2v", "ft", "bert", "codebert"]
embedding = embeddings[3]

Choose a project among Chrome and Linux or the full dataset

In [4]:
#data = pd.read_csv(os.path.join('..','data', 'full_data_reduced.csv'))
data = pd.read_csv(os.path.join('..','data', 'chrome_data_reduced.csv'))
#data = pd.read_csv(os.path.join('..','data', 'linux_data_reduced.csv'))

Shuffle the dataset before starting operating on it

In [5]:
data = data.sample(frac=1, random_state=seed).reset_index(drop=True)

In [6]:
print(data.head())

                                                func  vul  length
0    midimanagerusbtest() : message_loop_(new bas...    0      14
1  void renderframehostimpl::cancelblockedrequest...    0       8
2  void renderthreadimpl::removeobserver(renderth...    0       7
3  void webstorestandaloneinstaller::initinstalld...    0       7
4  omniboxstate::omniboxstate(const omniboxeditmo...    0      15


In [7]:
np.max(data["length"])

80

In [8]:
word_counts = data["func"].apply(lambda x: len(x.split()))
max_length = word_counts.max()
print("Maximum number of words:", max_length)


Maximum number of words: 80


In [9]:
vc = data["vul"].value_counts()

print(vc)

print("Vulnerability Percentage: ", (vc[1] / vc[0])*100, '%')

0    70232
1     3000
Name: vul, dtype: int64
Vulnerability Percentage:  4.271557124957284 %


Train-val-test split

In [10]:
# split data into train and test and then train into train and val (90% train, 10% test and then 90% train and 10% val)
shuffle_seeders = [seed, 10, 15, 20, 25, 30, 35, 40, 45, 50]
shuffle_seeder = shuffle_seeders[0]

x_train_val, x_test, y_train_val, y_test = train_test_split(data["func"].tolist(), data["vul"].tolist(), stratify = data["vul"].tolist(), test_size=0.1, random_state=shuffle_seeder)

x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, stratify = y_train_val, test_size=0.1, random_state=shuffle_seeder)


<b>Handling imbalanced data</b>

Class weighting

In [11]:
class_weights = {0:1, 1:1}
#class_weights = {0:len(data) / (len(vc) * vc[0]), 1:len(data) / (len(vc) * vc[1])} #total observations / (number of classes * observations in class)
#class_weights = dict(enumerate(compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)))
print(class_weights)

{0: 1, 1: 1}


Under-sampling of the clean samples

In [12]:
x_train = np.array(x_train)
x_train = x_train.reshape(-1, 1)
x_train.shape

(59317, 1)

In [13]:
# Apply under-sampling with the specified strategy
class_counts = pd.Series(y_train).value_counts()
print("Class distribution ", class_counts)
majority_class = class_counts.idxmax()
print("Majority class ", majority_class)
target_count = int(class_counts.iloc[0] / 2) 
print("Targeted number of majority class", target_count)

# under
sampling_strategy = {majority_class: target_count}        
rus = RandomUnderSampler(random_state=seed, sampling_strategy=sampling_strategy)
x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train) 
print("Class distribution after under-sampling", pd.Series(y_train_resampled).value_counts())

Class distribution  0    56887
1     2430
dtype: int64
Majority class  0
Targeted number of majority class 28443
Class distribution after under-sampling 0    28443
1     2430
dtype: int64


Synthetic Minority Over-sampling of the vulnerable samples

In [14]:
model_variation = "microsoft/codebert-base-mlm" # "neulab/codebert-cpp"
tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=False) # , do_lower_case=True
codebert = TFAutoModel.from_pretrained(model_variation) # , from_pt=True

Some layers from the model checkpoint at microsoft/codebert-base-mlm were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base-mlm.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [23]:
sentences = x_train_resampled.tolist()

sequences = [tokenizer.encode(sente[0], truncation=True, max_length=512, padding="max_length", add_special_tokens=False, return_tensors="tf") for sente in sentences]
embeddings = [codebert(sequence)[0][:, 0, :].numpy() for sequence in sequences]  # Extract embeddings from CodeBERT's output
extracted_embeddings = [arr[0] for arr in embeddings]

In [24]:
smote = SMOTE(random_state=seed, sampling_strategy='auto')  # 'auto' means it will resample to have the same number of samples as the majority class
x_train_resampled, y_train_resampled = smote.fit_resample(extracted_embeddings, y_train_resampled)

# Class distribution after SMOTE
class_counts_after_smote = pd.Series(y_train_resampled).value_counts()
print("Class distribution after SMOTE", class_counts_after_smote)

Class distribution after SMOTE 0    28443
1    28443
dtype: int64


Adaptive Synthetic Sampling with ADASYN - Over-sampling of the vulnerable samples

In [ ]:
# adasyn = ADASYN(random_state=seed, sampling_strategy='auto')  # 'auto' means it will resample to have the same number of samples as the majority class
# x_train_resampled, y_train_resampled = adasyn.fit_resample(extracted_embeddings, y_train_resampled)

# # Class distribution after ADASYN
# class_counts_after_adasyn = pd.Series(y_train_resampled).value_counts()
# print("Class distribution after SMOTE", class_counts_after_adasyn)

In [25]:
# Shuffle the resampled data while preserving the correspondence between features and labels
x_train_resampled, y_train_resampled = shuffle(x_train_resampled, y_train_resampled, random_state=seed)

# rename
X_train = x_train_resampled
Y_train = y_train_resampled

In [36]:
# codebert encoding also in test/val set
sequences_val = [tokenizer.encode(sente, truncation=True, max_length=512, padding="max_length", add_special_tokens=False, return_tensors="tf") for sente in x_val]
embeddings_val = [codebert(sequence)[0][:, 0, :].numpy() for sequence in sequences_val]  # Extract embeddings from CodeBERT's output
extracted_embeddings_val = [arr[0] for arr in embeddings_val]

In [87]:
X_val = extracted_embeddings_val
Y_val = y_val

In [86]:
dim = len(X_train[0])
dim

768

Functions for Deep Learnig

In [60]:
# Evaluation functions
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

def f2_loss(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float32)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    #tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f2 = 5*p*r / (4*p+r+K.epsilon())
    f2 = tf.where(tf.math.is_nan(f2), tf.zeros_like(f2), f2)
    
    return 1 - K.mean(f2)

In [61]:
from tensorflow.keras.layers import Layer

# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

def buildLstmWithBahdanauAttention(max_len, top_words, dim, seed, embedding_matrix, learning_rate=0.001):
    model = Sequential()
    kernel_initializer = glorot_uniform()
    model.add(Embedding(input_dim=top_words, output_dim=dim, input_length=max_len, weights=[embedding_matrix], mask_zero=True, trainable=False))
    model.add(GRU(500, activation='tanh', dropout=0.2, return_sequences=True, stateful=False, kernel_constraint=max_norm(3), bias_constraint=max_norm(3), kernel_initializer=kernel_initializer)) 

    model.add(GRU(100, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    model.add(GRU(200, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())

    # Add BahdanauAttention layer before the final output layer
    model.add(attention())

    model.add(Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer))

    optimizer = optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[f2_metric])
    return model


In [115]:
# Deep Learning Models - Classifiers
def buildLstm(dim, seed, learning_rate=0.001):
    model = Sequential()
    kernel_initializer = glorot_uniform()

    # LSTM/GRU Layers
    model.add(GRU(500, activation='tanh', dropout=0.2, return_sequences=True, stateful=False, input_shape=(1, dim), kernel_constraint=max_norm(3), bias_constraint=max_norm(3), kernel_initializer=kernel_initializer))
    model.add(GRU(100, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    model.add(GRU(200, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    
    model.add(BatchNormalization())
    
    # Fully Connected Layers
    #model.add(Dense(64, activation='relu', kernel_initializer=kernel_initializer))
    # Classification layer
    model.add(Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer))

    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[f2_metric])

    return model


def buildCnn(max_len, top_words, dim, seed, embedding_matrix):
    cnn_model = Sequential()
    cnn_model.add(Embedding(top_words, dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    '''cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))'''
    cnn_model.add(GlobalMaxPool1D())
    #cnn_model.add(Dense(units = 128, activation = 'relu'))
    cnn_model.add(Dense(units = 1, activation = 'sigmoid'))
    cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=[f2_metric])
    return cnn_model

def buildLstmCNN(dim, seed, learning_rate=0.001):
    model = Sequential()
    kernel_initializer = glorot_uniform()

    # LSTM/GRU Layers
    model.add(GRU(500, activation='tanh', dropout=0.2, return_sequences=True, stateful=False, input_shape=(1, dim), kernel_constraint=max_norm(3), bias_constraint=max_norm(3), kernel_initializer=kernel_initializer))
    model.add(GRU(100, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    model.add(GRU(200, activation='tanh', dropout=0.1, return_sequences=True, stateful=False, kernel_initializer=kernel_initializer))
    
    model.add(BatchNormalization())
    
    # CNN Layers
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_initializer=kernel_initializer)) 
    model.add(GlobalMaxPool1D())

    # Fully Connected Layers
    #model.add(Dense(64, activation='relu', kernel_initializer=kernel_initializer))
    # Classification layer
    model.add(Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer))

    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[f2_metric])

    return model


Training

In [ ]:
y_train = np.array(Y_train)
y_val = np.array(Y_val)

y_train = np.expand_dims(y_train, axis=1)
y_val = np.expand_dims(y_val, axis=1)

x_train = np.array(X_train)
x_val = np.array(X_val)

x_train = np.expand_dims(x_train, axis=1)
x_val = np.expand_dims(x_val, axis=1)

In [116]:
nb_epoch = 100
BS = 64
print("Training...")
milli_sec1 = int(round(time.time() * 1000))

userModel = "lstm"

if userModel == "cnn":
    myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix) 
elif userModel == "lstm":
    myModel = buildLstm(dim, seed)
elif userModel == "lstmCnn":
    myModel = buildLstmCNN(dim, seed)
elif userModel == "lstmAtt":
    myModel = buildLstmWithBahdanauAttention(max_len, num_words, dim, seed, embedding_matrix)
    
print("model summary\m", myModel.summary())
csv_logger = CSVLogger('log.csv', append=True, separator=',')
es = EarlyStopping(monitor='val_f2_metric', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_f2_metric', mode='max', verbose=1, save_best_only=True)

history = myModel.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = nb_epoch, batch_size = BS, shuffle=True, verbose=1, callbacks=[csv_logger,es,mc], class_weight=class_weights)

#load best model
#model = load_model('best_model.h5')
myModel.load_weights("best_model.h5")

scores = myModel.evaluate(x_val, y_val, verbose=0)
#predictions = myModel.predict_classes(X_test, verbose=0)
predScores = myModel.predict(x_val)
predictions = (predScores > 0.5).astype("int32")

accuracy=accuracy_score(y_val, predictions)
precision=precision_score(y_val, predictions)
recall=recall_score(y_val, predictions)
f1=f1_score(y_val, predictions)
roc_auc=roc_auc_score(y_val, predictions)
f2=5*precision*recall / (4*precision+recall)

cm = confusion_matrix(y_val, predictions, labels=[0, 1])
#print(cm)
sns.heatmap(cm, annot=True)
tn, fp, fn, tp = cm.ravel()

print("TP=",tp)
print("TN=",tn)
print("FP=",fp)
print("FN=",fn)

acc = ((tp+tn)/(tp+tn+fp+fn))

print("Accuracy:%.2f%%"%(acc*100))
print("Precision:%.2f%%"%(precision*100))
print("Recall:%.2f%%"%(recall*100))
print("F1 score:%.2f%%"%(f1*100))
print("Roc_Auc score:%.2f%%"%(roc_auc*100))
print("F2 score:%.2f%%"%(f2*100))
print(classification_report(y_val, predictions))

milli_sec2 = int(round(time.time() * 1000))
print("Cross Validation is completed after", milli_sec2-milli_sec1)

Training...


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_11. Consider increasing the input size. Received input shape [None, 1, 200] which would produce output shape with a zero or negative value in a dimension.